In [8]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    accounts = data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




{'name': '現金', 'category': '資産', 'sub_category': '流動資産'}
{'name': '建物', 'category': '資産', 'sub_category': '固定資産'}
{'name': '減価償却費', 'category': '費用', 'sub_category': '営業費用'}
{'name': '減価償却累計額', 'category': '資産', 'sub_category': '固定資産'}
{'name': '固定資産売却益', 'category': '収益', 'sub_category': '営業外収益'}
{'name': '固定資産売却損', 'category': '費用', 'sub_category': '営業外費用'}
{'name': '売上高', 'category': '収益', 'sub_category': '営業収益'}
{'name': '仕入', 'category': '費用', 'sub_category': '営業費用'}
{'name': '棚卸資産', 'category': '資産', 'sub_category': '流動資産'}
{'name': '売上原価', 'category': '費用', 'sub_category': '営業費用'}
{'name': '借入金', 'category': '負債', 'sub_category': '固定負債'}
{'name': '借入金', 'category': '負債', 'sub_category': '流動負債'}
{'name': '利益剰余金', 'category': '純資産', 'sub_category': '剰余金'}
{'name': '資本金', 'category': '純資産', 'sub_category': '資本金等'}


In [1]:
import pandas as pd

from player import *
# サンプルコード
game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
player2 = Player("player2",game_master)

product_A = player1.redister_product(name="Product_A")

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)
player1.sale_product(product_A, 80, 200)
player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)
player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A, 5)

end_1 = ledger1.execute_settlement()

ledger1.display_transaction_history()
ledger1.display_trial_balance(end_1)
ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")
df

在庫が 80 単位減少しました。総コスト: 12000
在庫が 100 単位減少しました。総コスト: 12600
在庫が 100 単位減少しました。総コスト: 12000


取引一覧:

  [ゲーム内時間]
    仕訳: 現金: 5000, 資本金: -5000
    摘要: Initial capital
  [ゲーム内時間]
    仕訳: 仕入: 15000, 現金: -15000
    摘要: 商品の仕入れ　商品名：Product_A 個数：100 単価：150
  [ゲーム内時間]
    仕訳: 仕入: 24015, 現金: -24015
    摘要: 商品の仕入れ　商品名：Product_A 個数：200 単価：120
  [ゲーム内時間]
    仕訳: 現金: 16000, 売上高: -16000
    摘要: 商品の売上 商品名：Product_A 個数：80 単価：200
  [ゲーム内時間]
    仕訳: 仕入: 4800, 現金: -4800
    摘要: 商品の仕入れ　商品名：Product_A 個数：30 単価：160
  [ゲーム内時間]
    仕訳: 現金: 22500, 売上高: -22500
    摘要: 商品の売上 商品名：Product_A 個数：100 単価：225
  [ゲーム内時間]
    仕訳: 現金: 20500, 売上高: -20500
    摘要: 商品の売上 商品名：Product_A 個数：100 単価：210
  [ゲーム内時間]
    仕訳: 売上原価: 33780, 仕入: -43815, 棚卸資産: 10035
    摘要: 棚卸調整 商品: Product_A


残高試算表:

現金: 20185
建物: 0
減価償却費: 0
減価償却累計額: 0
固定資産売却益: 0
固定資産売却損: 0
売上高: 0
仕入: 0
棚卸資産: 10035
売上原価: 0
借入金: 0
利益剰余金: (25220)
資本金: (5000)
純損益: 0

=== 貸借対照表 ===
資産:
  流動資産:
        現金: 20185
        棚卸資産: 10035
  固定資産:
  繰延資産:
負債:
  流動負債:
  固定負債:
純資産:
  株主資本:
  

,0
現金,20185
建物,0
減価償却費,0
減価償却累計額,0
固定資産売却益,0
固定資産売却損,0
売上高,-59000
仕入,0
棚卸資産,10035
売上原価,33780


In [6]:
from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
product_A = game_master.construct_instance("inventory","product_A")


資産 'product_A' (ID: 696f14b1-7162-478c-90cd-b0ac7f375d35) が登録されました。
